# Function Calling with Chat API

본 튜토리얼을 통해 Friendli Serverless Endpoints의 Chat 모델을 활용하여 동적으로 함수 호출에 필요한 인자를 생성하고, 이를 활용하여 함수를 호출하여 정보를 얻는 방법에 대해서 알아 봅시다.

Function calling이란 모델에게 사전에 정의한 함수의 정보(e.g., 함수 이름, 함수의 인자들, 함수의 설명)와 함께 질문을 했을 때, 그 질문에 알맞는 함수 호출을 할 수 있도록 모델이 함수의 인자 값을 생성해주는 기능을 말합니다. Function calling 기능은 다음과 같은 단계를 거쳐 활용이 가능합니다.

1. 함수의 인자를 생성
2. 생성된 함수 인자를 활용하여 함수 호출

함수 호출 결과는 이후 새로운 모델 답변을 생성할 때 입력으로 활용을 할 수 있습니다. 그러면 함수 호출을 통해 동적으로 생성한 정보를 답변 생성에 반영할 수 있습니다. 이런 점에서 function calling은 Retrieval Augmented Generation (RAG)에 활용 가능한 기법이라고 볼 수 있습니다. 즉, 질문의 의도가 반영된 동적인 함수 호출을 통해 정보를 가져오는 부분이 벡터 데이터베이스에서 질문에 해당하는 정보를 가져오는 부분과 비슷하다고 볼 수 있습니다.

본 튜토리얼에서는 사용자가 "지금 날씨에 어떤 옷을 입어야 해?"와 같은 질문을 했을 때 기상청 등에서 실시간으로 받아온 정보에 기반하여 옷차림을 추천해주는 간단한 예제를 다루어 보겠습니다.

## Installing Client

Friendli Serverless Endpoints는 OpenAI와 동일한 스키마의 Chat API를 제공합니다. Function calling 기능 역시 OpenAI Chat API와 동일한 옵션으로 사용이 가능합니다.
튜토리얼 진행을 위해 OpenAI Python 클라이언트를 설치합니다. 

In [ ]:
!pip install openai

## Defining Function

호출하고 싶은 함수를 정의합니다. 이 예제에서는 미리 정의된 입력(=지역)에 대해 정해진 결과(=기온)를 반환하는 dummy 함수를 정의하였습니다. 물론 실제 응용 사례에서는 날씨 서버에서 정보를 가져오는 함수를 정의해서 사용을 해야겠죠?

In [1]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "seoul" in location.lower():
        return 10
    elif "san francisco" in location.lower():
        return 72
    elif "paris" in location.lower():
        return 22
    else:
        return 20

## Sending Function Calling Request

이제 Chat API를 통해 함수 호출에 사용될 인자를 생성해봅시다.

우선 Friendli Serverless Endpoints에 API call 시 사용할 **Personal Access Token**을 준비합니다.

In [2]:
import getpass

friendli_token = getpass.getpass()

사용자가 오늘과 같은 날씨에 어떤 옷을 입어야 하는지 질문을 한다고 해봅시다.

In [3]:
question_from_user = "I live in Seoul. What should I wear for today's weather?"

OpenAI client를 사용해서 요청을 보냅니다.

- 이 때 사용자의 질문은 `messages` 옵션에 `user` role의 `content`로 들어갑니다.
- 위에서 정의했던 `get_current_weather` 함수의 정보는 `tools` 옵션에 들어갑니다. 이 때 `parameters`에는 위에서 정의했던 함수의 인자 정보가 [JSON Schema](https://json-schema.org/) 형식으로 제공해야 합니다.

In [4]:
from openai import OpenAI

client = OpenAI(base_url="https://inference.friendli.ai/v1", api_key=friendli_token)

messages = [
    {
        "role": "user",
        "content": question_from_user,
    },
]

tools=[
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 위에서 정의한 함수의 이름
            "description": "Get the current weather information in a given location.",  # 함수에 대한 설명
            "parameters": {  # JSON Schema로 정의한 함수의 인자 정보
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of current location e.g., Seoul",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature degree.",
                    },
                },
            },
        },
    },
]

chat = client.chat.completions.create(
    model="mixtral-8x7b-instruct-v0-1",
    messages=messages,
    tools=tools,
)
print(chat)

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_000', function=Function(arguments='{"location":"Seoul, South Korea, Asia","unit":"celsius"}', name='get_current_weather'), type='function')]))], created=1716474807, model=None, object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=137, total_tokens=157))


생성된 답변의 `arguments` 필드에 포함된 함수의 인자 값을 위에서 정의된 함수에 그대로 unpacking 하여 호출하면 사용자의 질문의 의도가 반영된 함수 호출 결과를 생성할 수 있습니다.

In [5]:
import json

func_kwargs = json.loads(chat.choices[0].message.tool_calls[0].function.arguments)

weather_info = get_current_weather(**func_kwargs)
print(weather_info)

10


함수 호출을 통해서 받아온 날씨 정보를 바탕으로 "I live in Seoul. What should I wear for today's weather?"에 대한 최종적인 답변을 생성 해봅시다.
이전 Chat API call에서 받아온 함수 인자 정보를 "assistant" 메세지로 추가하고, `get_current_weather` 함수를 호출해서 얻은 결과를 "tool" 메세지로 추가하여 다시 Chat API를 호출합니다. 

In [11]:
messages.append(
    {
        "role": "assistant",
        "tool_calls": [
            tool_call.dict()
            for tool_call in chat.choices[0].message.tool_calls
        ]
    }
)

messages.append(
    {
        "role": "tool",
        "content": str(weather_info),
        "tool_call_id": chat.choices[0].message.tool_calls[0].id
    }
)

chat_w_info = client.chat.completions.create(
    model="mixtral-8x7b-instruct-v0-1",
    tools=tools,
    messages=messages,
)
print(chat_w_info.choices[0].message.content)

Based on the current weather information, the temperature in Seoul, South Korea is 10 degrees Celsius.

Here is a suggestion for what to wear:

* A light jacket or sweater
* A long sleeve shirt
* A pair of long pants
* Socks and closed-toe shoes

It's also a good idea to bring a hat, gloves and scarf in case it gets colder. You may want to check the current weather information again before heading out as the weather can change throughout the day.
